# Sparse PCA Simulation

Let $$S_{p\times p}=\alpha[1\ \cdots\ 1\ 0\ \cdots\ 0]^T[1\ \cdots\ 1\ 0\ \cdots\ 0] + \beta I_p$$ and $$(Z_{n\times p})_{i, j}\sim\mathcal{N}(0, 1).$$
We define $$X_{n,p}\equiv Z_{n,p}S_{p,p}.$$

In [47]:
p = 50
n = 30
alpha = 0.8
beta = 0.2
half = np.empty((p, 1))
for i in range(p):
    if i < p / 2:
        half[i][0] = 1
    else:
        half[i][0] = 0
S = alpha * half @ half.T + beta * np.identity(p)

Z = np.empty((n, p))
for i in range(n):
    for j in range(p):
        Z[i][j] = np.random.normal(0, 1)
        
X = Z @ S

S, Z, X = torch.from_numpy(S), torch.from_numpy(Z), torch.from_numpy(X)

In [48]:
S.shape, Z.shape, X.shape

(torch.Size([50, 50]), torch.Size([30, 50]), torch.Size([30, 50]))

## Method 1
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-u(v\odot w)^T\rVert_F^2$$
where for any matrix $A$, 
#### $$\lVert A\rVert_F^2 = \sum_{i,j}A_{i,j}^2.$$

In [49]:
u = torch.randn(n, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [50]:
def objective_function_1(u, v, w):
    A = X - u @ (v * w).T
    return torch.sum(A * A)

In [51]:
epochs = 10000
step_size = 1e-4
for i in range(epochs):
    objective = objective_function_1(u, v, w)
    if i % 100 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_1(u, v, w))))

Epoch 0: loss = 12531.849196069039
Epoch 100: loss = 9949.311762782188
Epoch 200: loss = 1685.3677596809773
Epoch 300: loss = 67.45428886046383
Epoch 400: loss = 58.712289454231936
Epoch 500: loss = 56.379031922584694
Epoch 600: loss = 55.56337210584272
Epoch 700: loss = 55.233889866884056
Epoch 800: loss = 55.08586726916071
Epoch 900: loss = 55.01343788809092
Epoch 1000: loss = 54.97517655609602
Epoch 1100: loss = 54.95333992373527
Epoch 1200: loss = 54.93978034319419
Epoch 1300: loss = 54.930552641537304
Epoch 1400: loss = 54.923664304383166
Epoch 1500: loss = 54.91808474195901
Epoch 1600: loss = 54.913259825323344
Epoch 1700: loss = 54.90889082635339
Epoch 1800: loss = 54.904817264362535
Epoch 1900: loss = 54.900951623201564
Epoch 2000: loss = 54.897250257386695
Epoch 2100: loss = 54.89368913994254
Epoch 2200: loss = 54.890264497254144
Epoch 2300: loss = 54.88697146694304
Epoch 2400: loss = 54.88382123636567
Epoch 2500: loss = 54.880816763857894
Epoch 2600: loss = 54.87796595520854


In [52]:
print(v * w)

tensor([[-2.5999e+00],
        [-2.6020e+00],
        [-2.5918e+00],
        [-2.6145e+00],
        [-2.5843e+00],
        [-2.5852e+00],
        [-2.5929e+00],
        [-2.5715e+00],
        [-2.6145e+00],
        [-2.5732e+00],
        [-2.5954e+00],
        [-2.5712e+00],
        [-2.5880e+00],
        [-2.5969e+00],
        [-2.5736e+00],
        [-2.6000e+00],
        [-2.5993e+00],
        [-2.5890e+00],
        [-2.5708e+00],
        [-2.6462e+00],
        [-2.6372e+00],
        [-2.5707e+00],
        [-2.5905e+00],
        [-2.6047e+00],
        [-2.5854e+00],
        [-6.5733e-02],
        [ 3.9174e-02],
        [-1.7721e-02],
        [-1.7977e-03],
        [-2.5326e-02],
        [ 5.7989e-03],
        [ 5.0709e-03],
        [-7.5018e-02],
        [-8.7435e-03],
        [-2.0453e-02],
        [-2.6136e-02],
        [-3.8352e-02],
        [-2.3329e-03],
        [-1.6943e-02],
        [ 2.1261e-03],
        [-3.3205e-02],
        [-2.0841e-02],
        [ 9.1775e-03],
        [ 1

## Method 2
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-Xu(v\odot w)^T\rVert_F^2$$
where for any matrix $A$, 
#### $$\lVert A\rVert_F^2 = \sum_{i,j}A_{i,j}^2.$$

In [53]:
u = torch.randn(p, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [54]:
def objective_function_2(u, v, w):
    A = X - (X.double() @ u.double()).double() @ (v * w).T.double()
    return torch.sum(A * A)

In [56]:
epochs = 100000
step_size = 5e-7
for i in range(epochs):
    objective = objective_function_2(u, v, w)
    if i % 1000 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
#         if (i == epochs - 1):
#             print(u.grad)
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_2(u, v, w))))

Epoch 0: loss = 393755.65688339435
Epoch 1000: loss = 11360.322056002256
Epoch 2000: loss = 10721.354545773716
Epoch 3000: loss = 8875.45036000723
Epoch 4000: loss = 5290.025397770484
Epoch 5000: loss = 2684.5840019199923
Epoch 6000: loss = 1355.2058397678152
Epoch 7000: loss = 536.7391809569532
Epoch 8000: loss = 275.08155567543656
Epoch 9000: loss = 232.40000055631037
Epoch 10000: loss = 212.36223493295066
Epoch 11000: loss = 197.05772638228802
Epoch 12000: loss = 184.4937751356619
Epoch 13000: loss = 173.88780481238487
Epoch 14000: loss = 164.7704485928345
Epoch 15000: loss = 156.8282280488006
Epoch 16000: loss = 149.83791968983104
Epoch 17000: loss = 143.63358379309472
Epoch 18000: loss = 138.0897032605307
Epoch 19000: loss = 133.10741621846216
Epoch 20000: loss = 128.60855504350926
Epoch 21000: loss = 124.52835623232706
Epoch 22000: loss = 120.81468413532616
Epoch 23000: loss = 117.42313497912858
Epoch 24000: loss = 114.31500158422774
Epoch 25000: loss = 111.45925849137414
Epoch 2

In [57]:
print(v * w)

tensor([[ 1.6611e-01],
        [ 1.6628e-01],
        [ 1.6548e-01],
        [ 1.6706e-01],
        [ 1.6523e-01],
        [ 1.6520e-01],
        [ 1.6572e-01],
        [ 1.6436e-01],
        [ 1.6707e-01],
        [ 1.6426e-01],
        [ 1.6577e-01],
        [ 1.6421e-01],
        [ 1.6531e-01],
        [ 1.6588e-01],
        [ 1.6439e-01],
        [ 1.6608e-01],
        [ 1.6601e-01],
        [ 1.6542e-01],
        [ 1.6421e-01],
        [ 1.6904e-01],
        [ 1.6849e-01],
        [ 1.6433e-01],
        [ 1.6546e-01],
        [ 1.6641e-01],
        [ 1.6517e-01],
        [ 4.1107e-03],
        [-2.4657e-03],
        [ 1.1743e-03],
        [ 4.2167e-05],
        [ 1.6227e-03],
        [-3.0285e-04],
        [-3.4156e-04],
        [ 4.8365e-03],
        [ 5.4926e-04],
        [ 1.3867e-03],
        [ 1.6337e-03],
        [ 2.4697e-03],
        [ 1.3997e-04],
        [ 1.1410e-03],
        [-1.0170e-04],
        [ 2.0759e-03],
        [ 1.3488e-03],
        [-6.2382e-04],
        [-9

## Method 3
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-u(v\odot v - w\odot w)^T\rVert_F^2$$
where for any matrix $A$, 
#### $$\lVert A\rVert_F^2 = \sum_{i,j}A_{i,j}^2.$$

In [58]:
u = torch.randn(n, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [59]:
def objective_function_3(u, v, w):
    A = X - u @ (v * v - w * w).T
    return torch.sum(A * A)

In [60]:
epochs = 10000
step_size = 1e-4
for i in range(epochs):
    objective = objective_function_3(u, v, w)
    if i % 100 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_3(u, v, w))))

Epoch 0: loss = 15167.744033085686
Epoch 100: loss = 417.0710573238376
Epoch 200: loss = 55.65584797327891
Epoch 300: loss = 55.07225027346823
Epoch 400: loss = 54.97455633637689
Epoch 500: loss = 54.93394523295083
Epoch 600: loss = 54.91237281141591
Epoch 700: loss = 54.899082963626164
Epoch 800: loss = 54.890039115640434
Epoch 900: loss = 54.88342300666987
Epoch 1000: loss = 54.878320071507815
Epoch 1100: loss = 54.87422262038271
Epoch 1200: loss = 54.87083530363676
Epoch 1300: loss = 54.86798246403083
Epoch 1400: loss = 54.865545586295994
Epoch 1500: loss = 54.8634478673261
Epoch 1600: loss = 54.861627872628674
Epoch 1700: loss = 54.86005397597193
Epoch 1800: loss = 54.85868625616017
Epoch 1900: loss = 54.85750178296105
Epoch 2000: loss = 54.85647795381118
Epoch 2100: loss = 54.85559660001215
Epoch 2200: loss = 54.854838227995906
Epoch 2300: loss = 54.85418977972691
Epoch 2400: loss = 54.85363897182377
Epoch 2500: loss = 54.853169735839174
Epoch 2600: loss = 54.852775190295475
Epoch

In [61]:
print(v * v - w * w)

tensor([[ 2.7482e+00],
        [ 2.7503e+00],
        [ 2.7396e+00],
        [ 2.7636e+00],
        [ 2.7317e+00],
        [ 2.7326e+00],
        [ 2.7408e+00],
        [ 2.7181e+00],
        [ 2.7636e+00],
        [ 2.7200e+00],
        [ 2.7433e+00],
        [ 2.7178e+00],
        [ 2.7356e+00],
        [ 2.7449e+00],
        [ 2.7204e+00],
        [ 2.7483e+00],
        [ 2.7476e+00],
        [ 2.7366e+00],
        [ 2.7174e+00],
        [ 2.7971e+00],
        [ 2.7876e+00],
        [ 2.7173e+00],
        [ 2.7382e+00],
        [ 2.7532e+00],
        [ 2.7328e+00],
        [ 6.9477e-02],
        [-4.1434e-02],
        [ 1.8729e-02],
        [ 1.9023e-03],
        [ 2.6772e-02],
        [-6.1285e-03],
        [-5.3610e-03],
        [ 7.9293e-02],
        [ 9.2430e-03],
        [ 2.1619e-02],
        [ 2.7628e-02],
        [ 4.0537e-02],
        [ 2.4640e-03],
        [ 1.7907e-02],
        [-2.2488e-03],
        [ 3.5093e-02],
        [ 2.2028e-02],
        [-9.6992e-03],
        [-1

## Method 4
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-Xu(v\odot v - w\odot w)^T\rVert_F^2$$
where for any matrix $A$, 
#### $$\lVert A\rVert_F^2 = \sum_{i,j}A_{i,j}^2.$$

In [62]:
u = torch.randn(p, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [63]:
def objective_function_4(u, v, w):
    A = X - X.double() @ u.double() @ (v * v - w * w).T.double()
    return torch.sum(A * A)

In [64]:
epochs = 100000
step_size = 5e-7
for i in range(epochs):
    objective = objective_function_4(u, v, w)
    if i % 100 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_4(u, v, w))))

Epoch 0: loss = 3418886.0717749973
Epoch 100: loss = 12804.092525825186
Epoch 200: loss = 12143.751395911038
Epoch 300: loss = 11517.484607907862
Epoch 400: loss = 10875.54776541851
Epoch 500: loss = 10188.236517454903
Epoch 600: loss = 9449.365693758282
Epoch 700: loss = 8672.740890662255
Epoch 800: loss = 7880.928771065258
Epoch 900: loss = 7095.691033390601
Epoch 1000: loss = 6335.346819054968
Epoch 1100: loss = 5615.578351846615
Epoch 1200: loss = 4949.220328385312
Epoch 1300: loss = 4343.841031105189
Epoch 1400: loss = 3799.416204256683
Epoch 1500: loss = 3309.535466851905
Epoch 1600: loss = 2866.2800170828414
Epoch 1700: loss = 2465.1805689542084
Epoch 1800: loss = 2107.14593241608
Epoch 1900: loss = 1796.8413861180786
Epoch 2000: loss = 1538.6245915213058
Epoch 2100: loss = 1332.1348416310354
Epoch 2200: loss = 1170.6090435502656
Epoch 2300: loss = 1043.6878048488702
Epoch 2400: loss = 942.2654792492228
Epoch 2500: loss = 860.8903430937698
Epoch 2600: loss = 796.3056900147756
Ep

Epoch 22400: loss = 85.08961015211983
Epoch 22500: loss = 84.98162519488493
Epoch 22600: loss = 84.87449327152034
Epoch 22700: loss = 84.76843651059966
Epoch 22800: loss = 84.66326635024141
Epoch 22900: loss = 84.55897984997489
Epoch 23000: loss = 84.45565497655002
Epoch 23100: loss = 84.35326050540631
Epoch 23200: loss = 84.25180336651538
Epoch 23300: loss = 84.15114154938448
Epoch 23400: loss = 84.05143645192967
Epoch 23500: loss = 83.95245104523357
Epoch 23600: loss = 83.85415717319097
Epoch 23700: loss = 83.75666537159746
Epoch 23800: loss = 83.66004518264005
Epoch 23900: loss = 83.56427117907178
Epoch 24000: loss = 83.46936457982295
Epoch 24100: loss = 83.3753324922646
Epoch 24200: loss = 83.28208610007498
Epoch 24300: loss = 83.18951809521528
Epoch 24400: loss = 83.09765689739955
Epoch 24500: loss = 83.00650003000372
Epoch 24600: loss = 82.91605175146846
Epoch 24700: loss = 82.82637835922634
Epoch 24800: loss = 82.73740563177036
Epoch 24900: loss = 82.64915530636497
Epoch 25000: 

Epoch 45100: loss = 72.64188519195717
Epoch 45200: loss = 72.61348913047811
Epoch 45300: loss = 72.58520174631659
Epoch 45400: loss = 72.55700693276523
Epoch 45500: loss = 72.52892311821005
Epoch 45600: loss = 72.50091415984281
Epoch 45700: loss = 72.47302822249615
Epoch 45800: loss = 72.44526222581892
Epoch 45900: loss = 72.41756591009158
Epoch 46000: loss = 72.38994437258799
Epoch 46100: loss = 72.36241667709022
Epoch 46200: loss = 72.33507953211814
Epoch 46300: loss = 72.30787961949251
Epoch 46400: loss = 72.28083210116166
Epoch 46500: loss = 72.25386790119416
Epoch 46600: loss = 72.22696688445558
Epoch 46700: loss = 72.2001754854967
Epoch 46800: loss = 72.17352406088796
Epoch 46900: loss = 72.14696529293622
Epoch 47000: loss = 72.12049944343846
Epoch 47100: loss = 72.09411154328957
Epoch 47200: loss = 72.06781803181866
Epoch 47300: loss = 72.0416178735423
Epoch 47400: loss = 72.01553863549013
Epoch 47500: loss = 71.98955209832585
Epoch 47600: loss = 71.96364801526482
Epoch 47700: l

Epoch 66900: loss = 68.29263133641646
Epoch 67000: loss = 68.27853999996123
Epoch 67100: loss = 68.2644781346857
Epoch 67200: loss = 68.25048126458569
Epoch 67300: loss = 68.23652607139887
Epoch 67400: loss = 68.22260936532125
Epoch 67500: loss = 68.20873646196351
Epoch 67600: loss = 68.19489681585458
Epoch 67700: loss = 68.18109655119639
Epoch 67800: loss = 68.16733018776543
Epoch 67900: loss = 68.15358606215614
Epoch 68000: loss = 68.13986543113354
Epoch 68100: loss = 68.12616340977269
Epoch 68200: loss = 68.11247970488725
Epoch 68300: loss = 68.09881478013907
Epoch 68400: loss = 68.08517308731007
Epoch 68500: loss = 68.07155481799884
Epoch 68600: loss = 68.05798430423462
Epoch 68700: loss = 68.04445138808589
Epoch 68800: loss = 68.03094939045003
Epoch 68900: loss = 68.01748541488165
Epoch 69000: loss = 68.00408674866736
Epoch 69100: loss = 67.9907395872612
Epoch 69200: loss = 67.97741155840885
Epoch 69300: loss = 67.96410100565225
Epoch 69400: loss = 67.95080867558339
Epoch 69500: l

Epoch 89700: loss = 65.80061641803354
Epoch 89800: loss = 65.79217235133854
Epoch 89900: loss = 65.78373993279978
Epoch 90000: loss = 65.77531565117367
Epoch 90100: loss = 65.76689949009267
Epoch 90200: loss = 65.75849146801575
Epoch 90300: loss = 65.75009183748058
Epoch 90400: loss = 65.74170101703339
Epoch 90500: loss = 65.73331840427323
Epoch 90600: loss = 65.72496071620233
Epoch 90700: loss = 65.71663475261707
Epoch 90800: loss = 65.70832147793183
Epoch 90900: loss = 65.70002728293994
Epoch 91000: loss = 65.69174365747139
Epoch 91100: loss = 65.68346802870988
Epoch 91200: loss = 65.67520119701594
Epoch 91300: loss = 65.66694758567749
Epoch 91400: loss = 65.65870554768665
Epoch 91500: loss = 65.65047785398933
Epoch 91600: loss = 65.6422587180461
Epoch 91700: loss = 65.63404739579083
Epoch 91800: loss = 65.62584391767847
Epoch 91900: loss = 65.61765852990186
Epoch 92000: loss = 65.609511823098
Epoch 92100: loss = 65.6013766566934
Epoch 92200: loss = 65.59325083007056
Epoch 92300: los

In [65]:
print(v * v - w * w)

tensor([[ 1.0311e-01],
        [ 1.0325e-01],
        [ 1.0288e-01],
        [ 1.0377e-01],
        [ 1.0251e-01],
        [ 1.0255e-01],
        [ 1.0281e-01],
        [ 1.0196e-01],
        [ 1.0370e-01],
        [ 1.0200e-01],
        [ 1.0294e-01],
        [ 1.0199e-01],
        [ 1.0265e-01],
        [ 1.0302e-01],
        [ 1.0208e-01],
        [ 1.0313e-01],
        [ 1.0309e-01],
        [ 1.0264e-01],
        [ 1.0189e-01],
        [ 1.0492e-01],
        [ 1.0450e-01],
        [ 1.0197e-01],
        [ 1.0269e-01],
        [ 1.0325e-01],
        [ 1.0254e-01],
        [ 2.5862e-03],
        [-1.5486e-03],
        [ 6.4174e-04],
        [ 2.8849e-05],
        [ 9.9337e-04],
        [-2.8324e-04],
        [-2.2304e-04],
        [ 2.9616e-03],
        [ 3.2759e-04],
        [ 7.5451e-04],
        [ 1.0493e-03],
        [ 1.4802e-03],
        [ 9.7513e-05],
        [ 6.4638e-04],
        [-1.4642e-04],
        [ 1.3753e-03],
        [ 7.8028e-04],
        [-3.8168e-04],
        [-5